In [27]:
!gunzip ecoli_400K_err_1.fastq.gz
!gunzip ecoli_400K_err_2.fastq.gz

### 1. Оценка качества исправления ошибок
Исправить риды с помощью любой программы, и приложив исправленные и исходные риды к геному, оценить следующие значения:

Undetected error (false negative)

Detected & corrected error (true positive)

Detected and removed error (true positive)

Falsely corrected error (false positive)

Correctly unmodified base (true negative)

Incorrectly removed base (false positive)

Исправим риды с помощью `trimmomatic`. Возьмем риды с качеством не ниже 35.

In [36]:
!java -jar trimmomatic-0.39.jar PE ecoli_400K_err_1.fastq ecoli_400K_err_2.fastq output_forward_paired.fq.gz output_forward_unpaired.fq.gz output_reverse_paired.fq output_reverse_unpaired.fq LEADING:35 TRAILING:35

TrimmomaticPE: Started with arguments:
 ecoli_400K_err_1.fastq ecoli_400K_err_2.fastq output_forward_paired.fq.gz output_forward_unpaired.fq.gz output_reverse_paired.fq output_reverse_unpaired.fq LEADING:35 TRAILING:35
Quality encoding detected as phred33
Input Read Pairs: 1381602 Both Surviving: 1365886 (98,86%) Forward Only Surviving: 11888 (0,86%) Reverse Only Surviving: 2824 (0,20%) Dropped: 1004 (0,07%)
TrimmomaticPE: Completed successfully


Исправим риды с помощью `SPAdes`

In [55]:
!gunzip ecoli_400K_err_1.00.0_0.cor.fastq.gz
!gunzip ecoli_400K_err_2.00.0_0.cor.fastq.gz

In [ ]:
!./spades.py --only-error-correction --pe1-1 ecoli_400K_err_1.fastq --pe1-2 ecoli_400K_err_2.fastq --only-error-correсtion -o DATA

In [62]:
!gunzip ecoli_400K_err_2.fastq.00.0_0.cor.fastq.gz

In [38]:
!bwa index MG1655-K12.first400K.fasta > /dev/null 2>&1
!bwa mem MG1655-K12.first400K.fasta ecoli_400K_err_1.fastq 1> alignment1.sam 2> /dev/null
!bwa mem MG1655-K12.first400K.fasta output_forward_paired.fq 1> alignment2.sam 2> /dev/null

In [37]:
!gunzip output_forward_paired.fq.gz 

In [56]:
!bwa mem MG1655-K12.first400K.fasta ecoli_400K_err_1.00.0_0.cor.fastq 1> alignment3.sam 2> /dev/null

In [63]:
!bwa mem MG1655-K12.first400K.fasta ecoli_400K_err_2.00.0_0.cor.fastq 1> alignment4.sam 2> /dev/null

In [39]:
!samtools flagstat alignment1.sam

1381602 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
1378283 + 0 mapped (99.76% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [40]:
!samtools flagstat alignment2.sam

1365886 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
1344357 + 0 mapped (98.42% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [61]:
!samtools flagstat alignment3.sam

1374807 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
1367303 + 0 mapped (99.45% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


Для подсчета ошибок будем смотреть на каждую позицию в референсе и выравненные на нее позиции в ридах: в сырых данных и в исправленных.

Далее, если и в сырых данных, и в исправленных один и тот же нуклеотид и при этом он не совпадает с нуклеотидом из референса, то это *Undetected error*.

Если в сырых данных нуклеотид не равен нуклеотиду в референсе, но при этом в исправленных данных равен нуклеотиду в референсе, то это *Detected & corrected error*.

Если в исправленных данных нуклеотид отсутствует и отстутсвует в референсе, но при этом есть в сырых данных, то это *Detected and removed error*, если же в исправленных данных нуклеотид отсутствует, но при этом есть в сырых данных и в референсе, то это *Incorrectly removed base*.

Если в сырых данных нуклеотид совпадает с референсом, но в исправленных не совпадает, то это *Falsely corrected error*.

И наконец, если в сырых данных нуклеотид совпадает с референсом и совпадает с нуклеотидом в исправленных данных, то это *Correctly unmodified base*.

In [41]:
!samtools view -S -b alignment1.sam > alignment1.bam
!samtools sort alignment1.bam > aligment1_sorted.bam
!samtools index aligment1_sorted.bam

In [42]:
!samtools view -S -b alignment2.sam > alignment2.bam
!samtools sort alignment2.bam > aligment2_sorted.bam
!samtools index aligment2_sorted.bam

In [3]:
import pysam
from Bio import SeqIO
from tqdm import tqdm

Сначала посчитаем ошибки в ридах, исправленных с помощью `trimmomatic`

In [6]:
align_raw = pysam.AlignmentFile('aligment1_sorted.bam')
align_corr = pysam.AlignmentFile('aligment2_sorted.bam')
reference = SeqIO.read('MG1655-K12.first400K.fasta', 'fasta')

В словаре `errors` будем хранить типы ошибок и их количество

In [11]:
def errors_counter(align_raw, align_corr, reference):
    
    errors = {'Undetected errors' : 0,
          'Detected & corrected errors' : 0,
          'Detected and removed errors' : 0,
          'Incorrectly removed bases' : 0,
          'Falsely corrected errors' : 0,
          'Correctly unmodified bases' : 0
         }

    for alig1, alig2 in tqdm(zip(align_corr.pileup(), align_raw.pileup())):
        
        ref_nucl = reference[alig2.reference_pos]
        reads = {}

        for corr in alig1.pileups:
            
            if corr.query_position is not None:
                reads[corr.alignment.query_name] = corr.alignment.query_sequence[corr.query_position]
            
            else:
                reads[corr.alignment.query_name] = ''

        for raw in alig2.pileups:
            
            if raw.alignment.query_name in reads:
                
                if raw.query_position:
                    
                    raw_nucl  = raw.alignment.query_sequence[raw.query_position]
                    corr_nucl = reads[raw.alignment.query_name]
                    
                    if (raw_nucl == corr_nucl) and (corr_nucl != ref_nucl):
                        errors['Undetected errors'] += 1

                    if (raw_nucl == ref_nucl) and (corr_nucl != ref_nucl):
                        errors['Falsely corrected errors'] += 1

                    if (raw_nucl != ref_nucl) and (corr_nucl == ref_nucl):
                        errors['Detected & corrected errors'] += 1

                    if (raw_nucl == ref_nucl) and (corr_nucl == ref_nucl):
                        errors['Correctly unmodified bases'] += 1
            else:
                if raw.query_position:
                    
                    raw_nucl  = raw.alignment.query_sequence[raw.query_position]
                    
                    if (raw_nucl == ref_nucl):
                        errors['Incorrectly removed bases'] += 1
                        
                    else:
                        errors['Detected and removed errors'] += 1

    return errors

In [17]:
error_trimm = errors_counter(align_raw, align_corr, reference)

399999it [17:56, 371.64it/s]


In [18]:
error_trimm

{'Undetected errors': 116967,
 'Detected & corrected errors': 33,
 'Detected and removed errors': 50509,
 'Incorrectly removed bases': 8077242,
 'Falsely corrected errors': 36,
 'Correctly unmodified bases': 116414658}

Теперь посчитаем ошибки в исправлениях, сделанных с помощью `SPAdes`

In [57]:
!samtools view -S -b alignment3.sam > alignment3.bam
!samtools sort alignment3.bam > aligment3_sorted.bam
!samtools index aligment3_sorted.bam

In [14]:
align_corr_spades = pysam.AlignmentFile('aligment3_sorted.bam')

In [15]:
error_spades = errors_counter(align_raw, align_corr_spades, reference)

399999it [17:42, 376.51it/s]


In [16]:
error_spades

{'Undetected errors': 55895,
 'Detected & corrected errors': 100202,
 'Detected and removed errors': 11327,
 'Incorrectly removed bases': 1507573,
 'Falsely corrected errors': 82,
 'Correctly unmodified bases': 122984281}

In [64]:
!samtools view -S -b alignment4.sam > alignment4.bam
!samtools sort alignment4.bam > aligment4_sorted.bam
!samtools index aligment4_sorted.bam

In [65]:
align_corr_spades2 = pysam.AlignmentFile('aligment4_sorted.bam')

In [66]:
error_spades2 = errors_counter(align_raw, align_corr_spades2, reference)

399999it [36:14, 183.92it/s]


In [67]:
error_spades2 

{'Undetected errors': 126638,
 'Detected & corrected errors': 176,
 'Detected and removed errors': 91021812,
 'Incorrectly removed bases': 33463161,
 'Falsely corrected errors': 204,
 'Correctly unmodified bases': 47096}

Как видно, у `trimmomatic` гораздо больше ошибок типа  *Incorrectly removed bases*, поскольку исправления заключались лишь в удалении участков в отличие от `SPAdes`